In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import autograd
from torch.utils import data
from torch.backends import cudnn
import utils

import pandas as pd
import numpy as np

In [2]:
import utils.dataLoader
from utils.digitPrinter import print_digit
from utils.dataLoader import get_train_valid_loader

#### HYPERPARAMETER & CONFIGURATIONS

In [3]:
learning_rate = 0.01
max_epochs = 20
batch_size = 800

In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
cudnn.benchmark = True

#### NEURAL NET

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # three layers, input -> 250 -> 250 -> 10
        self.linear1 = nn.Linear(784, 250)
        self.linear2 = nn.Linear(250, 250)
        self.linear3 = nn.Linear(250, 250)
        self.linear4 = nn.Linear(250, 100)
        self.linear5 = nn.Linear(100, 100)
        self.linear6 = nn.Linear(100, 100)
        self.linear7 = nn.Linear(100, 50)
        self.linear8 = nn.Linear(50, 50)
        self.linear9 = nn.Linear(50, 25)
        self.linear10 = nn.Linear(25, 10)

    def forward(self, x):
        # activation functions between layers
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = F.relu(self.linear5(x))
        x = F.relu(self.linear6(x))
        x = F.relu(self.linear7(x))
        x = F.relu(self.linear8(x))
        x = F.relu(self.linear9(x))
        x = self.linear10(x)
        return F.log_softmax(x, dim=1)

#### READ DATA

In [6]:
train_loader, valid_loader = get_train_valid_loader("data/train.csv", batch_size, 10, 0.2)

In [7]:
# print_digit(valid_loader, 5)

### Initialize the net

In [8]:
# create NNet instance and initialize optimizer and criterion
net = Net()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
criterion = nn.NLLLoss()
print(net)

Net(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=250, bias=True)
  (linear3): Linear(in_features=250, out_features=250, bias=True)
  (linear4): Linear(in_features=250, out_features=100, bias=True)
  (linear5): Linear(in_features=100, out_features=100, bias=True)
  (linear6): Linear(in_features=100, out_features=100, bias=True)
  (linear7): Linear(in_features=100, out_features=50, bias=True)
  (linear8): Linear(in_features=50, out_features=50, bias=True)
  (linear9): Linear(in_features=50, out_features=25, bias=True)
  (linear10): Linear(in_features=25, out_features=10, bias=True)
)


In [9]:
train_loader, valid_loader = get_train_valid_loader("data/train.csv", batch_size, 10, 0.2)

In [10]:
# Loop over epochs
for epoch in range(max_epochs):
    # Training
    batch_idx = 0
    for local_train in train_loader:
        # Transfer to GPU
        local_batch = local_train['image']
        local_labels = local_train['label']
        
        # print(local_batch.shape)
        # print(local_labels.shape)
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        
        # Model computations
        optimizer.zero_grad()

        output = net(local_batch.float())
        loss = criterion(output, local_labels)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 5 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(local_batch), len(train_loader.sampler.indices),
                100. * batch_idx / len(train_loader), loss.data.item()))
        
        batch_idx += 1        

        
    # Validation
    with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for local_validate in valid_loader:
            # Transfer to GPU
            local_batch = local_validate['image']
            local_labels = local_validate['label']
        
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
                
            outputs = net(local_batch.float())
            _, predicted = torch.max(outputs.data, 1)
            total += len(local_labels)
            correct += (predicted == local_labels).sum().item()

        print('Accuracy of the network on validation images: %d %%' % (
            100 * correct / total))

Train Epoch: 0 [0/33600 (0%)]	Loss: 2.318320
Train Epoch: 0 [4000/33600 (12%)]	Loss: 2.309738
Train Epoch: 0 [8000/33600 (24%)]	Loss: 2.313692
Train Epoch: 0 [12000/33600 (36%)]	Loss: 2.308182
Train Epoch: 0 [16000/33600 (48%)]	Loss: 2.299282
Train Epoch: 0 [20000/33600 (60%)]	Loss: 2.306010
Train Epoch: 0 [24000/33600 (71%)]	Loss: 2.296736
Train Epoch: 0 [28000/33600 (83%)]	Loss: 2.292753
Train Epoch: 0 [32000/33600 (95%)]	Loss: 2.278688
Accuracy of the network on validation images: 17 %
Train Epoch: 1 [0/33600 (0%)]	Loss: 2.281184
Train Epoch: 1 [4000/33600 (12%)]	Loss: 2.262625
Train Epoch: 1 [8000/33600 (24%)]	Loss: 2.213019
Train Epoch: 1 [12000/33600 (36%)]	Loss: 2.163329
Train Epoch: 1 [16000/33600 (48%)]	Loss: 2.062081
Train Epoch: 1 [20000/33600 (60%)]	Loss: 2.009998
Train Epoch: 1 [24000/33600 (71%)]	Loss: 1.995417
Train Epoch: 1 [28000/33600 (83%)]	Loss: 1.886456
Train Epoch: 1 [32000/33600 (95%)]	Loss: 1.854651
Accuracy of the network on validation images: 29 %
Train Epoch: